In [ ]:
import numpy as np
import pandas as pd
import multiprocessing as mp
#** N is the number of nodes in the network; hardcoded
N = 7688059
# day is number of simulation day in the EpiHiper Output
day = 1001

#** repli is number of replicate in the EpiHiper output

repli = 60

#** defining state in the disease model
S = 1  #** susceptible node state
var1E = 2 #** variant-1 exposed node state
var1Isymp = 3 #** variant-1 symptomatic infected node state
var1Iasymp = 4 #** variant-1 asymptomatic infected node state
var2E = 5 #** variant-2 exposed node state
var2Isymp = 6 #** variant-1 symptomatic infected node state
var2Iasymp = 7 #**variant-1 asymptomatic infected node state
var1R=8
var2R=9
var1var2E=10
var1var2Isymp=11
var1var2Iasymp=12
var2var1E=13
var2var1Isymp=14
var2var1Iasymp=15
R = 16 #** recoverd state


tau = np.array([0.02,0.03])
seedLoc=np.array([51, 51059, 51121])
emergenceDay=np.array([90, 120, 150])
crossImmu=np.array([0.25, 0.5, 0.75, 1])
newVar = 1.4 #** relative transmisibility of new variant

colNames=["tau","seedLoc","emergenceDay","crossImmu","repli",'tick', 'S[current]',
       'S[in]', 'S[out]', 'var1R[current]', 'var1R[in]', 'var1R[out]',
       'var2R[current]', 'var2R[in]', 'var2R[out]', 'var1E[current]',
       'var1E[in]', 'var1E[out]', 'var1Isymp[current]', 'var1Isymp[in]',
       'var1Isymp[out]', 'var1Iasymp[current]', 'var1Iasymp[in]',
       'var1Iasymp[out]', 'var2E[current]', 'var2E[in]', 'var2E[out]',
       'var2Isymp[current]', 'var2Isymp[in]', 'var2Isymp[out]',
       'var2Iasymp[current]', 'var2Iasymp[in]', 'var2Iasymp[out]',
       'var1var2E[current]', 'var1var2E[in]', 'var1var2E[out]',
       'var1var2Isymp[current]', 'var1var2Isymp[in]', 'var1var2Isymp[out]',
       'var1var2Iasymp[current]', 'var1var2Iasymp[in]', 'var1var2Iasymp[out]',
       'var2var1E[current]', 'var2var1E[in]', 'var2var1E[out]',
       'var2var1Isymp[current]', 'var2var1Isymp[in]', 'var2var1Isymp[out]',
       'var2var1Iasymp[current]', 'var2var1Iasymp[in]', 'var2var1Iasymp[out]',
       'R[current]', 'R[in]', 'R[out]','V1SV1', 'V1SV2', 'V2SV1','V2SV2']
dpSummaryCombine=pd.DataFrame(columns = colNames)

combination=[(i, j, k, l) for i in tau for j in seedLoc for k in emergenceDay for l in crossImmu]
for i in combination:
    tau1=i[0]
    seedLoc1=i[1]
    emergenceDay1=i[2]
    crossImmu1=i[3]
    for it in np.arange(repli)+1:
        dp=pd.read_csv('/project/bii_nssac/COVID-19_USA_EpiHiper/rivanna/20210827-var_surv/exp7/tau_'+str(tau1)+'_var2_'+str(newVar)+'_seed2p_'+str(seedLoc1)+'_seed2t_'+str(emergenceDay1)+'_cross1by2_'+str('{:.2f}'.format(round(crossImmu1, 2)))+'/replicate_'+str(it)+'/output.csv.gz')
        dpSummary=pd.read_csv('/project/bii_nssac/COVID-19_USA_EpiHiper/rivanna/20210827-var_surv/exp7/tau_'+str(tau1)+'_var2_'+str(newVar)+'_seed2p_'+str(seedLoc1)+'_seed2t_'+str(emergenceDay1)+'_cross1by2_'+str('{:.2f}'.format(round(crossImmu1, 2)))+'/replicate_'+str(it)+'/outputSummary.csv.gz')
        #print(dpSummary)
        nR=len(dp.index)
        diseasePropagation = np.zeros((nR, 3),dtype = np.int)
        diseasePropagation[:, 0] = dp['tick']
        diseasePropagation[:, 1] = dp['pid']
        #diseasePropagation[dp['exit_state'] == "var1E", 2] = var1E
        diseasePropagation[dp['exit_state'] == "var1Isymp", 2] = var1Isymp
        diseasePropagation[dp['exit_state'] == "var1Iasymp", 2] = var1Iasymp
        #diseasePropagation[dp['exit_state'] == "var2E", 2] = var2E
        diseasePropagation[dp['exit_state'] == "var2Isymp", 2] = var2Isymp
        diseasePropagation[dp['exit_state'] == "var2Iasymp", 2] = var2Iasymp
        diseasePropagation[dp['exit_state'] == "var1R", 2] = var1R
        diseasePropagation[dp['exit_state'] == "var2R", 2] = var2R
        #diseasePropagation[dp['exit_state'] == "var1var2E", 2] = var1var2E
        #diseasePropagation[dp['exit_state'] == "var1var2Isymp", 2] = var1var2Isymp
        #diseasePropagation[dp['exit_state'] == "var1var2Iasymp", 2] = var1var2Iasymp
        #diseasePropagation[dp['exit_state'] == "var2var1E", 2] = var2var1E
        #diseasePropagation[dp['exit_state'] == "var2var1Isymp", 2] = var2var1Isymp
        #diseasePropagation[dp['exit_state'] == "var2var1Iasymp", 2] = var2var1Iasymp
        #diseasePropagation[dp['exit_state'] == "R", 2] = R
        #print(diseasePropagation)
    
    
        diseasePropagationIndex=np.zeros((day, 2),dtype = np.int)
    
        diseasePropagationIndex[:,0]=diseasePropagation[:,0].searchsorted(np.arange(day))
        diseasePropagationIndex[0:day-1,1]=diseasePropagationIndex[1:day,0]
        diseasePropagationIndex[day-1,1]=diseasePropagation[:,0].searchsorted(day)
        
        V1 = np.zeros((N, 1),dtype = np.int)
        V2 = np.zeros((N, 1),dtype = np.int)
        nS = np.zeros((N, 1),dtype = np.int)
        V1SV1 = np.zeros((N, 1),dtype = np.int)
        V2SV1 = np.zeros((N, 1),dtype = np.int)
        
        V1SV2 = np.zeros((N, 1),dtype = np.int)
        V2SV2 = np.zeros((N, 1),dtype = np.int)
        
        V1SV1prev = np.zeros((N, 1),dtype = np.int)
        V2SV1prev = np.zeros((N, 1),dtype = np.int)
        
        V1SV2prev = np.zeros((N, 1),dtype = np.int)
        V2SV2prev = np.zeros((N, 1),dtype = np.int)
        
        V1SV1sum = np.zeros((day, 1),dtype = np.int)
        V2SV1sum = np.zeros((day, 1),dtype = np.int)
        
        V1SV2sum = np.zeros((day, 1),dtype = np.int)
        V2SV2sum = np.zeros((day, 1),dtype = np.int)
        
        d = 1
        while (d <= 400):
            #nD1=diseasePropagation[(diseasePropagation[diseasePropagationIndex[d,0]:diseasePropagationIndex[d,1], 2]==var1Isymp)|(diseasePropagation[diseasePropagationIndex[d,0]:diseasePropagationIndex[d,1], 2]==var1Iasymp),1]
            V1SV1=V1SV1prev
            V1SV2=V1SV1prev
            V2SV2=V1SV1prev
            V2SV1=V1SV1prev
            diseasePropagationDay=diseasePropagation[diseasePropagationIndex[d,0]:diseasePropagationIndex[d,1],:]
            #print(diseasePropagationDay)
            if len(diseasePropagationDay[(diseasePropagationDay[:,2]==var1Isymp)|(diseasePropagationDay[:,2]==var1Iasymp),1])>0:
                diseasePropagationDay1=diseasePropagationDay[(diseasePropagationDay[:,2]==var1Isymp)|(diseasePropagationDay[:,2]==var1Iasymp),:]
                #print(diseasePropagationDay1)
                #print()
                V1SV1[diseasePropagationDay1[(V1[diseasePropagationDay1[:,1]]>V2[diseasePropagationDay1[:,1]]).flatten(),1]]=1
                V1SV2[diseasePropagationDay1[(V1[diseasePropagationDay1[:,1]]<V2[diseasePropagationDay1[:,1]]).flatten(),1]]=1
                
                V1[diseasePropagationDay1[:,1]]=d
                
            if len(diseasePropagationDay[(diseasePropagationDay[:,2]==var2Isymp)|(diseasePropagationDay[:,2]==var2Iasymp),1])>0:
                diseasePropagationDay2=diseasePropagationDay[(diseasePropagationDay[:,2]==var2Isymp)|(diseasePropagationDay[:,2]==var2Iasymp),:]
                V2SV1[diseasePropagationDay2[(V1[diseasePropagationDay2[:,1]]>V2[diseasePropagationDay2[:,1]]).flatten(),1]]=1
                V2SV2[diseasePropagationDay2[(V1[diseasePropagationDay2[:,1]]<V2[diseasePropagationDay2[:,1]]).flatten(),1]]=1
                
                V2[diseasePropagationDay2[:,1]]=d
            #if (diseasePropagationDay[diseasePropagationDay[:,2]==var1Isymp]):
#             for r in np.arange((diseasePropagationIndex[d,1]-diseasePropagationIndex[d,0]))+diseasePropagationIndex[d,0]:
#                 if (diseasePropagation[r,2]==var1Isymp)|(diseasePropagation[r,2]==var1Iasymp):
                    
#                     if (V1[diseasePropagation[r,1]]==0)&(V2[diseasePropagation[r,1]]==0):
#                         V1[diseasePropagation[r,1]]=d
#                     elif V1[diseasePropagation[r,1]]>V2[diseasePropagation[r,1]]:
#                         V1[diseasePropagation[r,1]]=d
#                         V1SV1[diseasePropagation[r,1]]=1
#                     else:
#                         V1[diseasePropagation[r,1]]=d
#                         V1SV2[diseasePropagation[r,1]]=1
                    
#                 if (diseasePropagation[r,2]==var2Isymp)|(diseasePropagation[r,2]==var2Iasymp):
#                     if (V1[diseasePropagation[r,1]]==0)&(V2[diseasePropagation[r,1]]==0):
#                         V2[diseasePropagation[r,1]]=d
#                     elif V1[diseasePropagation[r,1]]>V2[diseasePropagation[r,1]]:
#                         V2[diseasePropagation[r,1]]=d
#                         V2SV1[diseasePropagation[r,1]]=1
#                     else:
#                         V2[diseasePropagation[r,1]]=d
#                         V2SV2[diseasePropagation[r,1]]=1

#                 if (diseasePropagation[r,2]==var1R):
#                     V1SV1[diseasePropagation[r,1]]=0
#                     V1SV2[diseasePropagation[r,1]]=0
#                 if (diseasePropagation[r,2]==var2R):
#                     V2SV1[diseasePropagation[r,1]]=0
#                     V2SV2[diseasePropagation[r,1]]=0
                if len(diseasePropagationDay[(diseasePropagationDay[:,2]==var1R),1])>0:
                    V1SV1[diseasePropagationDay[(diseasePropagationDay[:,2]==var1R).flatten()]]=0
                    V1SV2[diseasePropagationDay[(diseasePropagationDay[:,2]==var1R).flatten()]]=0
                if len(diseasePropagationDay[(diseasePropagationDay[:,2]==var2R),1])>0:
                    V2SV1[diseasePropagationDay[(diseasePropagationDay[:,2]==var2R).flatten()]]=0
                    V2SV2[diseasePropagationDay[(diseasePropagationDay[:,2]==var2R).flatten()]]=0
                
                V1SV1prev=V1SV1
                V1SV2prev=V1SV2
                V2SV1prev=V2SV1
                V2SV2prev=V2SV2
                
                V1SV1sum[d]=V1SV1.sum()
                V1SV2sum[d]=V1SV2.sum()
                V2SV1sum[d]=V2SV1.sum()
                V2SV2sum[d]=V2SV2.sum()
                    
                
            
            #print(d)
            
            d=d+1
        
        #print(V1SV1row)
        it2=np.zeros((len(dpSummary),1),dtype = np.int)
        it2=it
        dpSummary.insert(4, "repli", it2, True)
        
        tau2=np.zeros((len(dpSummary),1),dtype = np.float)
        tau2=tau1
        dpSummary.insert(0, "tau", tau2, True)
        
        seedLoc2=np.zeros((len(dpSummary),1),dtype = np.float)
        seedLoc2=seedLoc1
        dpSummary.insert(1, "seedLoc", seedLoc2, True)
        
        emergenceDay2=np.zeros((len(dpSummary),1),dtype = np.float)
        emergenceDay2=emergenceDay1
        dpSummary.insert(2, "emergenceDay", emergenceDay2, True)
        
        crossImmu2=np.zeros((len(dpSummary),1),dtype = np.float)
        crossImmu2=crossImmu1
        dpSummary.insert(3, "crossImmu", crossImmu2, True)
        
        dpSummary.insert(54, "V1SV1", V1SV1sum, True)
        dpSummary.insert(55, "V1SV2", V1SV2sum, True)
        dpSummary.insert(56, "V2SV1", V2SV1sum, True)
        dpSummary.insert(57, "V2SV2", V2SV2sum, True)
        
        #print(dpSummary)
        dpSummaryCombine=dpSummaryCombine.append(dpSummary) 
        #print(it)
    #break
    print(tau1)
f='/project/biocomplexity/biosurveillance/EpiHiper/postProcessing/combineSamplingResult/'+'populationCountPyVVVVExp7'+'.csv'
dpSummaryCombine.to_csv(f, encoding='utf-8', index=False)


                                                                                                                                    
                                                                                                                                     
    

0.02
0.02
0.02


In [21]:
V1SV1[diseasePropagationDay1[(V1[diseasePropagationDay1[:,1]]>V2[diseasePropagationDay1[:,1]).flatten()],1]]

SyntaxError: closing parenthesis ')' does not match opening parenthesis '[' (<ipython-input-21-26f046c026cc>, line 1)

In [22]:
V1SV1[diseasePropagationDay1[(V1[diseasePropagationDay1[:,1]]>V2[diseasePropagationDay1[:,1]]).flatten(),1]]

array([], shape=(0, 1), dtype=int64)

In [23]:
V1SV1[3525951]

array([0])

In [17]:
V1[diseasePropagationDay1[:,1]]

array([[0]])

In [19]:
V1.shape

(7688059, 1)